In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import preprocessing

In [2]:
df_train = pd.read_csv('train_final.csv')
df_test = pd.read_csv('test_final.csv')
df_test = df_test.drop('ID',axis = 1)

In [3]:
#One Hot Encoding
for col in ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'sex', 'race', 'native.country']:
    one_hot_train = pd.get_dummies(df_train[col], prefix=col)
    one_hot_test = pd.get_dummies(df_test[col], prefix=col)

    df_train = df_train.drop(col,axis = 1)
    df_train = pd.concat([df_train,one_hot_train],axis=1)

    df_test = df_test.drop(col,axis = 1)
    df_test = pd.concat([df_test,one_hot_test],axis=1)

In [4]:
for col in df_test.columns:
    if col not in df_train.columns:
        df_train[col] = 0
        print(col)

native.country_Holand-Netherlands


In [5]:
X = df_train.loc[:, df_train.columns != 'income>50K']
y = df_train['income>50K']

X_test_final = df_test.loc[:, df_test.columns != 'income>50K']

In [6]:
estimator = SVC()

parameters = {
    'C': [5, 7],
    'gamma': [1,0.1,0.01],
    'kernel': ['rbf'],
    #'degree': [3],
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'accuracy',
    cv = 5,
    verbose=True
)

In [10]:
grid_search.fit(X,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [8]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_search.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 SVC(C=5)

 The best score across ALL searched params:
 0.8002800000000001

 The best parameters across ALL searched params:
 {'C': 5, 'kernel': 'rbf'}


In [8]:
best_params = {'C': 5, 'kernel': 'rbf'}

In [9]:
model = SVC(**best_params)
model.fit(X, y)

In [ ]:
y_pred = model.predict(X_test_final)

In [ ]:
import csv
ctr = 1
data = [['ID', 'Prediction']]
for pred in y_pred:
    row = [ctr, pred]
    data.append(row)
    ctr += 1
# open the file in the write mode
with open('results_svm.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)